# 대스타



1.   모듈화
2.   통합



#### 필요한 라이브러리 pip로 설치

In [1]:
pip install Pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#### 라이브러리 선언

In [2]:
#경로 접근을 위한 라이브러리
import os 

#이미지 출력을 위한 라이브러리
%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

import cv2
from google.colab.patches import cv2_imshow #colab에서만 사용하는 라이브러리, 이미지를 보여주기위한 것

#계산을 위한 라이브러리
import numpy as np

#Pillow 패키지 사용을 위한 라이브러리
from PIL import Image

#색공간 변환을 위한 라이브러리
from skimage import color

#csv 파일을 위한 라이브러리
import csv

#pandas 사용을 위한 라이브러리
import pandas as pd

#### dataset directory path

In [3]:
#기본 경로 설정
base_dir = '/content/drive/MyDrive'

#학습 데이터 경로
train_dir = os.path.join(base_dir, 'train') #원본 이미지 경로

re_train_dir = os.path.join(base_dir, 're_train') #resize된 이미지가 저장될 경로 : 실제 사용할 경로

#레이블 경로 설정
lable_dir = os.path.join(base_dir, 'train_label.csv') #원본 레이블 경로

print(train_dir)
print(re_train_dir)

/content/drive/MyDrive/train
/content/drive/MyDrive/re_train


#### 레이블 


> Pandas 사용



In [4]:
#pandas를 이용하여 csv파일 읽어온다
lable_input = pd.read_csv(filepath_or_buffer=lable_dir, encoding="cp949", sep=",") #한글 인코딩 : cp949 / 안되면 utf-8 사용해보기

# 항목 수（행수 * 칼럼수）를 반환한다.
print(lable_input.size)

#현재 저장되어 있는 정보
print(lable_input)

4164
           file_name    name
0     train_0000.jpg    네귀쓴풀
1     train_0001.jpg    네귀쓴풀
2     train_0002.jpg   민박쥐나물
3     train_0003.jpg   민박쥐나물
4     train_0004.jpg  큰틈성이삭새
...              ...     ...
2077  train_2077.jpg      뚝갈
2078  train_2078.jpg      뚝갈
2079  train_2079.jpg      뚝갈
2080  train_2080.jpg      뚝갈
2081  train_2081.jpg      뚝갈

[2082 rows x 2 columns]


#### 필요한 함수 구현하기

##### 전체 데이터 관련

###### 현재 전체 데이터 개수 + 1

In [5]:
def cnt_data():
  return len(lable_input)

###### 현재 가지고 있는 종자 리스트

In [6]:
def data_list():
  # row 생략 없이 출력
  pd.set_option('display.max_rows', None)
  #위의 생략 없이를 하지 않으면 index 범위 오류가 발생한다.

  #중복 제거
  tmp = lable_input.drop_duplicates(subset='name').iloc[:,1]

  #종자 개수
  n = len(tmp)

  name_list_tmp = str(tmp).split()
  name_list = []

  for i in range(n):
    name_list.append(name_list_tmp[2 * i + 1])

  return name_list

###### 이미지 개수가 n개인 종자 리스트 구하기

In [7]:
def n_list(n):
  num = []
  for i in data_list():
    tmp = img_cnt(i)
    if tmp == n:
      num.append(i)

  return num

##### 한 종자에 관련

###### 종자가 가진 이미지 수 구하기

*parameter*

> img_name : 종자 이름

In [8]:
def img_cnt(img_name):
  tmp = lable_input[lable_input['name'] == img_name].iloc[:,0]
  return len(tmp)

###### 종자가 가진 이미지 이름 리스트 구하기

*parameter*

> img_name : 종자 이름

In [9]:
def img_list(img_name):
  tmp = lable_input[lable_input['name'] == img_name].iloc[:,0]
  #입력한 종자이름을 가진 이미지 수
  n = len(tmp)

  #문자열 변환 후 리스트에 저장
  file_list_tmp = str(tmp).split()
  file_list = []
  for i in range(n):
    file_list.append(file_list_tmp[2 * i+1])

  return file_list

##### 증강 기법

###### re-size

*parameter*

> val : 변경할 사이즈



In [10]:
def re_size(val):
  size = val, val #바꾸고 싶은 사이즈
  #저장될 경로 : re_train_dir

  #현재 가지고 있는 이미지 이름 리스트
  train_fnames = os.listdir(train_dir)
  
  for i in range(len(train_fnames)):
    #가져오는 경로
    img = os.path.join(train_dir, train_fnames[i])
    #image 읽어오기
    image = cv2.imread(img)

    #사이즈 변경하기
    mod_img = cv2.resize(image,size)

    #저장할 경로 설정
    path = os.path.join(re_train_dir, train_fnames[i])
    cv2.imwrite(path, mod_img)

###### rotation

*parameter*

> n : 기본적으로 가지고 있는 이미지 수

In [11]:
def rotation(n):
  for i in n: #종자이름 반복문
    tmp = img_list(i)
    for j in tmp: #이미지 파일 이름 반복문
      #image path 저장
      images = os.path.join(re_train_dir, j)

      #image 읽어오기
      image = cv2.imread(images) 

      #회전시키기
      img90 = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE) # 시계방향으로 90도 회전
      img180 = cv2.rotate(image, cv2.ROTATE_180) # 180도 회전
      img270 = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE) # 반시계방향으로 90도 회전 = 시계방향으로 270도 회전

      #이미지 이름생성
      n = cnt_data() #현재 마지막 이미지 번호 +1
      t1 = f'train_{n}.jpg' #img90 이름 
      t2 = f'train_{n+1}.jpg' #img180 이름 
      t3 = f'train_{n+2}.jpg' #img270 이름 

      #수정된 학습 폴더에 90 이미지 저장
      path = os.path.join(re_train_dir, t1)
      cv2.imwrite(path, img90)
      #pandas에 추가 --> 맨 뒤에 이미지 이름, 종자이름 추가
      lable_input.loc[n] = [t1,i]

      #수정된 학습 폴더에 180 이미지 저장
      path = os.path.join(re_train_dir, t2)
      cv2.imwrite(path, img180)
      #pandas에 추가 --> 맨 뒤에 이미지 이름, 종자이름 추가
      lable_input.loc[n+1] = [t2,i]

      #수정된 학습 폴더에 270 이미지 저장
      path = os.path.join(re_train_dir, t3)
      cv2.imwrite(path, img270)
      #pandas에 추가 --> 맨 뒤에 이미지 이름, 종자이름 추가
      lable_input.loc[n+2] = [t3,i]

###### 이미지 선명하게

*parameter*

> n : 기본적으로 가지고 있는 이미지 수

In [12]:
def sharp(n):
  # 커널 생성(대상이 있는 픽셀을 강조)
  kernel = np.array([[0, -1, 0],
                    [-1, 5, -1],
                    [0, -1, 0]])
  
  for i in n: #종자 이름 반복
    tmp = img_list(i)
    for j in tmp: #이미지 파일 이름 반복
      #image path 저장
      images = os.path.join(re_train_dir, j)

      #image 읽어오기
      image = cv2.imread(images) 

      # 커널 적용 
      image_sharp = cv2.filter2D(image, -1, kernel)

      #이미지 이름생성
      n = cnt_data() #현재 마지막 이미지 번호 +1
      t = f'train_{n}.jpg'

      #수정된 학습 폴더에 sharp 이미지 저장
      path = os.path.join(re_train_dir, t)
      cv2.imwrite(path, image_sharp)
      #pandas에 추가 --> 맨 뒤에 이미지 이름, 종자이름 추가
      lable_input.loc[n] = [t,i]

###### 밝기 조절

In [13]:
def bright(n) :
  val = [30, 40, 50] # 밝기 조절 값

  for i in n: #종자이름 반복문
    tmp = img_list(i)
    for j in tmp: #이미지 파일 이름 반복문
      #image path 저장
      images = os.path.join(re_train_dir, j)

      #image 읽어오기
      image = cv2.imread(images) 

      #배열을 쓰는 이유 : RGB값 모두 더해줘야 하므로
      #배열을 쓰지 않으면 한 값만 커지거나 작아진다.
      array0 = np.full(image.shape, (val[0],val[0],val[0]), dtype=np.uint8)
      array1 = np.full(image.shape, (val[1],val[1],val[1]), dtype=np.uint8)
      array2 = np.full(image.shape, (val[2],val[2],val[2]), dtype=np.uint8)

      #더 밝게 하는 add
      add0 = cv2.add(image, array0) 
      add1 = cv2.add(image, array1)
      add2 = cv2.add(image, array2)

      #더 어둡게 하는 sub
      sub0 = cv2.subtract(image, array0)
      sub1 = cv2.subtract(image, array1)
      sub2 = cv2.subtract(image, array2)

      #이미지 이름생성
      n = cnt_data() #현재 마지막 이미지 번호 +1
      create_name = [] #생성한 이름
      for cn in range(len(val) * 2):
        create_name.append(f'train_{n+cn}.jpg')

      #만들어진 이미지 저장
      path = os.path.join(re_train_dir, create_name[0])
      cv2.imwrite(path, add0)
      path = os.path.join(re_train_dir, create_name[1])
      cv2.imwrite(path, add1)
      path = os.path.join(re_train_dir, create_name[2])
      cv2.imwrite(path, add2)

      path = os.path.join(re_train_dir, create_name[3])
      cv2.imwrite(path, sub0)
      path = os.path.join(re_train_dir, create_name[4])
      cv2.imwrite(path, sub1)
      path = os.path.join(re_train_dir, create_name[5])
      cv2.imwrite(path, sub2)

      #만들어진 이미지 pandas 저장
      for cn in range(len(create_name)):
        lable_input.loc[n+cn] = [create_name[cn],i]

In [14]:
def bright_2(n) :
  val = [35, 45] # 밝기 조절 값

  for i in n: #종자이름 반복문
    tmp = img_list(i)
    for j in tmp: #이미지 파일 이름 반복문
      #image path 저장
      images = os.path.join(re_train_dir, j)

      #image 읽어오기
      image = cv2.imread(images) 

      #배열을 쓰는 이유 : RGB값 모두 더해줘야 하므로
      #배열을 쓰지 않으면 한 값만 커지거나 작아진다.
      array0 = np.full(image.shape, (val[0],val[0],val[0]), dtype=np.uint8)
      array1 = np.full(image.shape, (val[1],val[1],val[1]), dtype=np.uint8)

      #더 밝게 하는 add
      add0 = cv2.add(image, array0) 
      add1 = cv2.add(image, array1)

      #더 어둡게 하는 sub
      sub0 = cv2.subtract(image, array0)
      sub1 = cv2.subtract(image, array1)

      #이미지 이름생성
      n = cnt_data() #현재 마지막 이미지 번호 +1
      create_name = [] #생성한 이름
      for cn in range(len(val) * 2):
        create_name.append(f'train_{n+cn}.jpg')

      #만들어진 이미지 저장
      path = os.path.join(re_train_dir, create_name[0])
      cv2.imwrite(path, add0)
      path = os.path.join(re_train_dir, create_name[1])
      cv2.imwrite(path, add1)

      path = os.path.join(re_train_dir, create_name[2])
      cv2.imwrite(path, sub0)
      path = os.path.join(re_train_dir, create_name[3])
      cv2.imwrite(path, sub1)

      #만들어진 이미지 pandas 저장
      for cn in range(len(create_name)):
        lable_input.loc[n+cn] = [create_name[cn],i]

###### 대비 높이기

In [15]:
def eqHist(n) :
  for i in n: #종자이름 반복문
    tmp = img_list(i)
    for j in tmp: #이미지 파일 이름 반복문
      #image path 저장
      images = os.path.join(re_train_dir, j)

      #image 읽어오기
      image = cv2.imread(images) 

      #히스토그램 평활화(Histogram Equalization)은 객체의 형태가 두드러지도록 만들어주는 이미지 처리 도구
      #equalizeHist() 메소드 사용
      #컬러이미지 : RGB --> YUV --> RGB 포맷 변경 필요

      # YUV 컬로 포맷으로 변환
      image_yuv = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
      # 히스토그램 평활화 적용
      image_yuv[:, :, 0] = cv2.equalizeHist(image_yuv[:, :, 0])
      #RGB로 변환
      image_rgb = cv2.cvtColor(image_yuv, cv2.COLOR_YUV2RGB)

      #이미지 이름생성
      n = cnt_data() #현재 마지막 이미지 번호 +1
      t = f'train_{n}.jpg'

      #수정된 학습 폴더에 이미지 저장
      path = os.path.join(re_train_dir, t)
      cv2.imwrite(path, image_rgb)
      #pandas에 추가 --> 맨 뒤에 이미지 이름, 종자이름 추가
      lable_input.loc[n] = [t,i]

#### test

In [16]:
cnt_data() #시작 데이터 개수

2082

In [17]:
len(data_list()) #가지고 있는 종자 개수

309

###### 전처리

re_size : 256*256

[0000:2081]

In [18]:
#전체 이미지 re-size

re_size(256)

###### 이미지 2개를 가진 종자

2 --> 56 증강

In [19]:
two = n_list(2)
len(two) #이미지 2개를 가진 종자 개수

79

[2082:2555]

In [20]:
#rotation

rotation(two)

In [21]:
cnt_data() #rotation 후 데이터 개수 --> 474 증가

2556

[2556:6347]

In [22]:
#bright

bright(two)

In [23]:
cnt_data() #bright 후 데이터 개수 --> 3792 증가

6348

In [24]:
sharp(two)

In [25]:
for i in two:
  print(img_cnt(i))

112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112


##### 이미지 3개를 가진 종자

3 --> 84

In [26]:
three = n_list(3)
len(three)

76

[6348:7031]

In [27]:
rotation(three)

In [28]:
cnt_data() #rotation 후 데이터 개수 --> 684 증가

11456

[7032:12503]

In [29]:
bright(three)

In [30]:
cnt_data() #bright 후 데이터 개수 --> 5472 증가

16928

In [31]:
sharp(three)

In [32]:
for i in three:
  print(img_cnt(i))

168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168


##### 이미지 4개를 가진 종자

4 --> 112

In [33]:
four = n_list(4)
len(four)

17

[12504:12707]

In [34]:
rotation(four)

In [35]:
cnt_data() #rotation 후 데이터 개수 --> 204 증가

23516

[12708:14340]

In [36]:
bright(four)

In [37]:
cnt_data() #bright 후 데이터 개수 --> 1632 증가

25148

In [38]:
for i in four:
  print(img_cnt(i))

112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112


##### 이미지 5개를 가진 종자

~~17개가 아닌~~ 7개 있음 ~~(실행을 잘못 시켰었음)~~

5 --> 140

In [39]:
five = n_list(5)
len(five)

7

[14340:14444]

In [40]:
rotation(five)

In [41]:
cnt_data() #rotation 후 데이터 개수 --> 105 증가

25253

[14445:15284]

In [42]:
bright(five)

In [43]:
cnt_data() #bright 후 데이터 개수 --> 840 증가

26093

In [44]:
for i in five:
  print(img_cnt(i))

140
140
140
140
140
140
140


##### 이미지 6개를 가진 종자

6 --> 120

In [45]:
six = n_list(6)
len(six)

6

[15285:15392]

In [46]:
rotation(six)

In [47]:
cnt_data() #rotation 후 데이터 개수 --> 108 증가

26201

[15393:15968]

In [48]:
bright_2(six)

In [49]:
cnt_data() #bright_2 후 데이터 개수 --> 576 증가

26777

In [50]:
for i in six:
  print(img_cnt(i))

120
120
120
120
120
120


##### 이미지 7개를 가진 종자

7 --> 140

In [51]:
seven = n_list(7)
len(seven)

2

[15969:16010]

In [52]:
rotation(seven)

In [53]:
cnt_data() #rotation 후 데이터 개수 --> 46 증가

26819

[16011:16234]

In [54]:
bright_2(seven)

In [55]:
cnt_data() #bright_2 후 데이터 개수 --> 224 증가

27043

In [56]:
for i in seven:
  print(img_cnt(i))

140
140


##### 이미지 8개를 가진 종자

8 --> 160

In [57]:
eight = n_list(8)
len(eight)

11

~~실수로 print문 안써서 중간값 모름~~

colab환경 상 오류가 어쩔 수 없이 난다. 


> 오류시 대처를 위해 한 셀에 하나의 증강기법만 사용



[16235:17906]

In [58]:
rotation(eight)

cnt_data()

27307

In [59]:
bright_2(eight)

cnt_data() #bright_2 후 데이터 개수 --> 224 증가

28715

In [60]:
for i in eight:
  print(img_cnt(i))

160
160
160
160
160
160
160
160
160
160
160


##### 이미지 9개를 가진 종자

9 --> 180

[17907:18095] rotation

[18096:19104] bright_2

In [61]:
nine = n_list(9)
len(nine)

7

In [62]:
rotation(nine)

print(cnt_data())

28904


In [63]:
bright_2(nine)

print(cnt_data())

29912


In [64]:
for i in nine:
  print(img_cnt(i))

180
180
180
180
180
180
180


##### 이미지 10개를 가진 종자

10 --> 160

[19104:19445]

[19446:19565]

[19566:19806]

In [65]:
ten = n_list(10)
len(ten)

12

In [66]:
rotation(ten)

print(cnt_data())

30272


In [67]:
sharp(ten)

print(cnt_data())

30752


In [68]:
eqHist(ten)

print(cnt_data())

31712


In [69]:
for i in ten:
  print(img_cnt(i))

160
160
160
160
160
160
160
160
160
160
160
160


##### 이미지 11개를 가진 종자

11--> 176

[19806:20735]

[20736:21131]

[21132:21923]

In [70]:
ten = n_list(11) #ten은 10이상 모든 수를 나타냄 / 파라미터로 주어지는 숫자만 잘 확인하기!
len(ten)

29

In [71]:
rotation(ten)

print(cnt_data())

32669


In [72]:
sharp(ten)

print(cnt_data())

33945


In [73]:
eqHist(ten)

print(cnt_data())

36497


In [74]:
for i in ten:
  print(img_cnt(i))

176
176
176
176
176
176
176
176
176
176
176
176
176
176
176
176
176
176
176
176
176
176
176
176
176
176
176
176
176


##### 이미지 12개를 가진 종자

12 --> 192

[21924:21995]

[21996:22091]

[22092:22283]

In [75]:
#계속 오류나서 2번씩 돌리면 안뜬다. 그래서 따로 씀
ten = n_list(12) #ten은 10이상 모든 수를 나타냄 / 파라미터로 주어지는 숫자만 잘 확인하기!
len(ten)

2

In [76]:
rotation(ten) #계속 오류 1번씩 난다. 그냥 다시 돌리면 된다.

print(cnt_data())

36569


In [77]:
sharp(ten)

print(cnt_data())

36665


In [78]:
eqHist(ten)

print(cnt_data())

36857


In [79]:
for i in ten:
  print(img_cnt(i))

192
192


##### 이미지 13개를 가진 종자

13 --> 104

[22284:23219]

[23220:24467]

In [80]:
#계속 오류나서 2번씩 돌리면 안뜬다. 그래서 따로 씀
ten = n_list(13) #ten은 10이상 모든 수를 나타냄 / 파라미터로 주어지는 숫자만 잘 확인하기!
len(ten)

24

In [81]:
rotation(ten) #계속 오류 1번씩 난다. 그냥 다시 돌리면 된다.

print(cnt_data())

37793


In [82]:
sharp(ten)

print(cnt_data())

39041


In [83]:
for i in ten:
  print(img_cnt(i))

104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104
104


##### 이미지 14개를 가진 종자

14 --> 112

[24468:24551]

[24552:24664]

In [84]:
#계속 오류나서 2번씩 돌리면 안뜬다. 그래서 따로 씀
ten = n_list(14) #ten은 10이상 모든 수를 나타냄 / 파라미터로 주어지는 숫자만 잘 확인하기!
len(ten)

2

In [85]:
rotation(ten) #계속 오류 1번씩 난다. 그냥 다시 돌리면 된다.

print(cnt_data())

39125


In [86]:
sharp(ten)

print(cnt_data())

39237


In [87]:
for i in ten:
  print(img_cnt(i))

112
112


##### 이미지 15개를 가진 종자

15 --> 120

[24664:25518]

[25519:26658]

In [88]:
#계속 오류나서 2번씩 돌리면 안뜬다. 그래서 따로 씀
ten = n_list(15) #ten은 10이상 모든 수를 나타냄 / 파라미터로 주어지는 숫자만 잘 확인하기!
len(ten)

19

In [89]:
rotation(ten) #계속 오류 1번씩 난다. 그냥 다시 돌리면 된다.

print(cnt_data())

40092


In [90]:
sharp(ten)

print(cnt_data())

41232


In [91]:
for i in ten:
  print(img_cnt(i))

120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120
120


##### 이미지 16개를 가진 종자

16 --> 128

[26659:26898]

[26899:27218]

In [92]:
#계속 오류나서 2번씩 돌리면 안뜬다. 그래서 따로 씀
ten = n_list(16) #ten은 10이상 모든 수를 나타냄 / 파라미터로 주어지는 숫자만 잘 확인하기!
len(ten)

5

In [93]:
rotation(ten) #계속 오류 1번씩 난다. 그냥 다시 돌리면 된다.

print(cnt_data())

41472


In [94]:
sharp(ten)

print(cnt_data())

41792


In [95]:
for i in ten:
  print(img_cnt(i))

128
128
128
128
128


##### 이미지 17개를 가진 종자

17 --> 136


In [96]:
#계속 오류나서 2번씩 돌리면 안뜬다. 그래서 따로 씀
ten = n_list(17) #ten은 10이상 모든 수를 나타냄 / 파라미터로 주어지는 숫자만 잘 확인하기!
len(ten)

1

In [97]:
rotation(ten) #계속 오류 1번씩 난다. 그냥 다시 돌리면 된다.

print(cnt_data())

41843


In [98]:
sharp(ten)

print(cnt_data())

41911


In [99]:
for i in ten:
  print(img_cnt(i))

136


##### 이미지 18개를 가진 종자


In [100]:
#계속 오류나서 2번씩 돌리면 안뜬다. 그래서 따로 씀
ten = n_list(18) #ten은 10이상 모든 수를 나타냄 / 파라미터로 주어지는 숫자만 잘 확인하기!
len(ten)

2

In [101]:
rotation(ten) #계속 오류 1번씩 난다. 그냥 다시 돌리면 된다.

print(cnt_data())

42019


In [102]:
sharp(ten)

print(cnt_data())

42163


In [103]:
for i in ten:
  print(img_cnt(i))

144
144


##### 이미지 19개를 가진 종자

In [104]:
#계속 오류나서 2번씩 돌리면 안뜬다. 그래서 따로 씀
ten = n_list(19) #ten은 10이상 모든 수를 나타냄 / 파라미터로 주어지는 숫자만 잘 확인하기!
len(ten)

3

In [105]:
rotation(ten) #계속 오류 1번씩 난다. 그냥 다시 돌리면 된다.

print(cnt_data())

42334


In [106]:
sharp(ten)

print(cnt_data())

42562


In [107]:
for i in ten:
  print(img_cnt(i))

152
152
152


##### 이미지 20개를 가진 종자

In [108]:
#계속 오류나서 2번씩 돌리면 안뜬다. 그래서 따로 씀
ten = n_list(20) #ten은 10이상 모든 수를 나타냄 / 파라미터로 주어지는 숫자만 잘 확인하기!
len(ten)

2

In [109]:
rotation(ten) #계속 오류 1번씩 난다. 그냥 다시 돌리면 된다.

print(cnt_data())

42682


In [110]:
sharp(ten)

print(cnt_data())

42842


In [111]:
for i in ten:
  print(img_cnt(i))

160
160


##### 이미지 21개를 가진 종자

In [112]:
#계속 오류나서 2번씩 돌리면 안뜬다. 그래서 따로 씀
ten = n_list(21) #ten은 10이상 모든 수를 나타냄 / 파라미터로 주어지는 숫자만 잘 확인하기!
len(ten)

2

In [113]:
rotation(ten) #계속 오류 1번씩 난다. 그냥 다시 돌리면 된다.

print(cnt_data())

42968


In [114]:
sharp(ten)

print(cnt_data())

43136


In [115]:
for i in ten:
  print(img_cnt(i))

168
168


##### 이미지 32개를 가진 종자

In [116]:
#계속 오류나서 2번씩 돌리면 안뜬다. 그래서 따로 씀
ten = n_list(32) #ten은 10이상 모든 수를 나타냄 / 파라미터로 주어지는 숫자만 잘 확인하기!
len(ten)

1

In [117]:
rotation(ten) #계속 오류 1번씩 난다. 그냥 다시 돌리면 된다.

print(cnt_data())

43232


In [118]:
for i in ten:
  print(img_cnt(i))

128


##### pandas --> csv 저장

In [119]:
lable_input.to_csv('re_train.csv', encoding='cp949')